In [ ]:
pip install v20 scikit-learn ta oandapyV20

In [ ]:
import requests
import json
import os

access_token = os.environ["OANDA_LIVE_API_TOKEN"]
OANDA_BEARER_TOKEN = os.environ["OANDA_BEARER_TOKEN"]

# Define the API endpoint for fetching historical price data
instrument = "USD_MXN" # Replace with the instrument you're interested in
url = f"https://api-fxtrade.oanda.com/v3/instruments/{instrument}/candles"

# Define the headers for the API request
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {OANDA_BEARER_TOKEN}'
}

# Define the parameters for the API request (e.g., the granularity of the candles and the price type
params = {
    'count': 2000,  # The number of candles to fetch
    'granularity': 'W',  #  4H candles
    'price': 'M'  # M for midpoint prices
}

# Make the API request
response = requests.get(url, headers=headers, params=params)

# Parse the JSON response
data = json.loads(response.text)

# Print the data
print(json.dumps(data, indent=4))



In [ ]:
#CREATE THE DATA FRAME:
import pandas as pd
import ta

# Extract the candles data
candles = data['candles']

# Convert the candles data into a DataFrame
df = pd.DataFrame([{'o': float(candle['mid']['o']), 'h': float(candle['mid']['h']), 'l': float(candle['mid']['l']), 'c': float(candle['mid']['c'])} for candle in candles])

# Add a time column to the DataFrame
df['time'] = [candle['time'] for candle in candles]

# Convert the time column to datetime format
df['time'] = pd.to_datetime(df['time'])

# Set the time column as the index of the DataFrame
df.set_index('time', inplace=True)

# Calculate the Simple Moving Average for the closing prices
df['sma_10'] = ta.trend.sma_indicator(df['c'], window=10)  # 10-period SMA
df['sma_20'] = ta.trend.sma_indicator(df['c'], window=20)  # 20-period SMA

# Calculate the Relative Strength Index (RSI)
df['rsi'] = ta.momentum.rsi(df['c'], window=14)

# Calculate the Moving Average Convergence Divergence (MACD)
macd = ta.trend.MACD(df['c'], window_slow=26, window_fast=12)
df['macd'] = macd.macd()
df['macd_signal'] = macd.macd_signal()
df['macd_hist'] = macd.macd_diff()

# Drop the rows with NaN values
df = df.dropna()

# Print the DataFrame
print(df)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Define a function to create feature set and target variable
def create_dataset(df, history_size=50, target_size=1):
    data = []
    labels = []
    start_index = history_size
    end_index = len(df) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Include additional features in the data
        features = df[['c', 'sma_10', 'sma_20', 'rsi', 'macd', 'macd_signal', 'macd_hist']].iloc[indices].values
        data.append(features.flatten())  # Flatten the data
        labels.append(df['c'].iloc[i+target_size])
    return np.array(data), np.array(labels)

# Create feature set and target variable
X, y = create_dataset(df)

# Split the data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error of the predictions
mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
# Define a function to predict the next 4-hour bar
def predict_next(model, df, history_size=50):
    # Get the most recent data
    recent_data = df[['c', 'sma_10', 'sma_20', 'rsi', 'macd', 'macd_signal', 'macd_hist']].iloc[-history_size:].values
    # Reshape the data
    recent_data = recent_data.flatten().reshape(1, -1)
    # Make a prediction
    prediction = model.predict(recent_data)
    return prediction

# Predict the closing price for the next 4-hour bar
next_price = predict_next(model, df)
next_price

In [ ]:
# Import necessary libraries
from sklearn.metrics import r2_score
from datetime import datetime, timedelta

# Calculate R2 score
r2 = r2_score(y_test, y_pred)

# Get the date and time for the next 4-hour bar
next_time = df.index[-1] + timedelta(hours=4)

# Print the information
print(f'Prediction for the next 4-hour bar (at {next_time}): {next_price[0]}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2) score: {r2}')

In [ ]:
STOP
# Import necessary libraries
from sklearn.metrics import r2_score
from datetime import datetime, timedelta

# Calculate R2 score
r2 = r2_score(y_test, y_pred)

# Get the date and time for the next 4-hour bar
next_time = df.index[-1] + timedelta(hours=4)

# Print the information
print(f'Prediction for the next 4-hour bar (at {next_time}): {next_price[0]}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2) score: {r2}')

In [ ]:
import os 
OANDA_TOKEN_DEMO = os.environ["OANDA_TOKEN_DEMO"]
ACCOUNT_ID_DEMO = os.environ["ACCOUNT_ID_DEMO"]

In [ ]:
import os
import oandapyV20
from oandapyV20 import API
from oandapyV20.exceptions import V20Error, StreamTerminated
from oandapyV20.endpoints.pricing import PricingStream
from oandapyV20.endpoints.orders import OrderCreate
from pytz import timezone

# Import necessary libraries
from sklearn.metrics import r2_score
from datetime import datetime, timedelta

# Set up OANDA API credentials and account details
api_token = os.environ["OANDA_TOKEN_DEMO"]
account_id = os.environ["ACCOUNT_ID_DEMO"]

# Create an instance of the API
api = API(access_token=api_token)

# Function to retrieve the current price
def get_current_price():
    params = {
        "instruments": "EUR_USD",  # Replace with the desired instrument
    }
    r = PricingStream(accountID=account_id, params=params)
    rv = api.request(r)
    return rv['prices'][0]['closeoutBid']  # Assuming you want the bid price

# Function to place an order
def place_order(instrument, units, order_type):
    data = {
        "order": {
            "price": get_current_price(),
            "instrument": instrument,
            "units": units,
            "type": order_type,
            "timeInForce": "FOK"  # Replace with your desired time in force
        }
    }
    r = OrderCreate(accountID=account_id, data=data)
    rv = api.request(r)
    return rv

# Calculate R2 score
r2 = r2_score(y_test, y_pred)

# Get the date and time for the next 4-hour bar
next_time = df.index[-1].to_pydatetime() + timedelta(hours=4)
next_time = next_time.replace(tzinfo=timezone('UTC'))  # Set the timezone

# Print the information
print(f'Prediction for the next 4-hour bar (at {next_time}): {next_price[0]}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2) score: {r2}')

# Check the price every 4 hours and place an order
if datetime.now(timezone('UTC')) >= next_time:
    current_price = get_current_price()
    print(f'Current price: {current_price}')
    # Place your order here using place_order() function


